# Credit Card Fraud Analysis (SQL + Python)

## Project Overview
The aim of this project is to analyze credit card transaction data to identify patterns and statistics that differentiate **fraudulent** transactions from **non-fraudulent** ones.

Using SQL and Python:
1. Explore and segment transactions based on **amount**, **time of day**, and **risk level**
2. Uncover high-risk behaviors and time-based fraud trends
3. Provide business insights to guide **fraud detection** and **prevention strategies**

In [1]:
# Creating Database
import sqlite3, pandas as pd
from pathlib import Path

# creating database
DB_PATH = Path("creditcard.db")

conn = sqlite3.connect(DB_PATH)
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")
print("Connected to:", DB_PATH.resolve())

Connected to: /Users/divya/Desktop/projects/fraud detection/creditcard.db


In [2]:
# Importing CSV & Setting up
import pandas as pd
import sqlite3
df = pd.read_csv("creditcard.csv")
df.head()
conn = sqlite3.connect("creditcard.db")
df.to_sql("creditcard", conn, if_exists= "replace", index= False)
pd.set_option("display.float_format", lambda x: f"{x:.4f}")

In [3]:
# Count of Fraud vs Non-Fraud Transactions
count = pd.read_sql("""
SELECT COUNT(*) AS total_transactions,
SUM(CASE WHEN CLASS = 1 THEN 1 ELSE 0 END) as fraud_transaction,
SUM(CASE WHEN CLASS = 0 THEN 1 ELSE 0 END) as nonfraud_transcaction 
FROM creditcard;""", conn)
count

,total_transactions,fraud_transaction,nonfraud_transcaction
0,284807,492,284315


In [4]:
# % of Fraud Cases
percentage = pd.read_sql("""
SELECT
ROUND(SUM(Class = 1) * 100.0/ COUNT(*),3) AS fraud_percentage,
ROUND(SUM(Class = 0) * 100.0 / COUNT(*),3) AS nonfraud_percentage
FROM creditcard;""", conn)
percentage

,fraud_percentage,nonfraud_percentage
0,0.1730,99.8270


In [5]:
# Flag High-Value Transactions (more than 1000)
high_value = pd.read_sql("""
SELECT *,
CASE WHEN Amount >= 1000 THEN 1 ELSE 0 END AS high_value_flag
FROM creditcard
ORDER BY Amount DESC
LIMIT 10;""", conn)
high_value

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,Amount,Class,high_value_flag
0,166198.0000,-35.5485,-31.8505,-48.3256,15.3042,-113.7433,73.3016,120.5895,-27.3474,-3.8724,...,5.7123,-1.5811,4.5845,4.5547,3.4156,31.6122,-15.4301,25691.1600,0,1
1,48401.0000,-36.8023,-63.3447,-20.6458,16.7155,-20.6721,7.6940,24.9566,-4.7301,-2.6873,...,-10.9331,-17.1737,1.1807,-7.0258,-2.5343,-3.6025,3.4502,19656.5300,0,1
2,95286.0000,-34.5493,-60.4646,-21.3409,16.8753,-19.2291,6.3353,24.4227,-4.9646,0.1889,...,-9.4994,-16.5132,0.7443,-7.0813,-2.6046,-3.5510,3.2508,18910.0000,0,1
3,42951.0000,-23.7128,-42.1727,-13.3208,9.9250,-13.9455,5.5649,15.7106,-2.8443,-1.5807,...,-6.3207,-11.3103,0.4042,-4.5473,-1.5771,-2.3574,2.2537,12910.9300,0,1
4,46253.0000,-21.7807,-38.3053,-12.1225,9.7528,-12.8808,4.2560,14.7851,-2.8183,-0.6673,...,-5.6194,-10.5470,0.6532,-4.2324,-0.4805,-2.2579,2.0825,11898.0900,0,1
5,119713.0000,-20.9249,-37.9435,-14.0603,10.4730,-10.8666,6.2567,14.9605,-2.3922,-0.5971,...,-6.9264,-9.9287,-0.4471,-4.8482,-2.2416,-2.1407,2.0015,11789.8400,0,1
6,172273.0000,-9.0305,-11.1126,-16.2338,3.5920,-40.4277,23.9178,44.0545,-7.2778,-4.2106,...,0.9881,7.0400,0.3477,2.5209,2.3425,3.4782,-2.7131,10199.4400,0,1
7,145283.0000,-21.5325,-34.7048,-8.3030,10.2642,3.9572,-3.2297,-4.0668,-4.0840,0.5541,...,-7.3311,-32.8290,0.1190,-8.6966,-1.7781,-0.5198,2.7167,10000.0000,0,1
8,55709.0000,-16.9501,-16.4174,-12.5234,6.5556,-27.7530,18.0720,28.5041,-10.1522,2.1247,...,0.0501,-10.8559,1.5504,-0.5022,0.8217,12.1524,-4.0098,8790.2600,0,1
9,152763.0000,-14.6417,-28.5548,-12.7145,5.8783,-7.8551,2.4710,11.9226,-2.6512,-2.2240,...,-3.2697,-8.0246,0.4230,-2.3967,-0.6334,-1.7637,1.4220,8787.0000,0,1


In [6]:
# Fraud Rate by Value
value = pd.read_sql("""
SELECT 
CASE WHEN Amount >= 1000 THEN 'High Value' ELSE 'Normal Value' END AS highvalue_category,
  COUNT(*) AS total_transaction,
  SUM(Class) AS fraud_transaction,
  ROUND(SUM(Class) * 100.0 / COUNT(*), 2) AS fraud_rate_percentage
FROM creditcard
GROUP BY highvalue_category;""", conn)
value

,highvalue_category,total_transaction,fraud_transaction,fraud_rate_percentage
0,High Value,3069,9,0.2900
1,Normal Value,281738,483,0.1700


In [7]:
# Fraud Rate by Hour of day
hour = pd.read_sql( """
SELECT
FLOOR (Time/3600) AS hour,
COUNT(*) AS total_transactions,
ROUND(SUM(Class)*100.0/COUNT(*), 2) AS fraud_rate
FROM creditcard
GROUP BY hour
ORDER BY hour;""", conn)
hour

,hour,total_transactions,fraud_rate
0,0.0000,3963,0.0500
1,1.0000,2217,0.0900
2,2.0000,1576,1.3300
3,3.0000,1821,0.7100
4,4.0000,1082,0.5500
5,5.0000,1681,0.6500
6,6.0000,1831,0.1600
7,7.0000,3368,0.6800
8,8.0000,5179,0.1000
9,9.0000,7878,0.1900


In [8]:
# Fraud Rate in the diff parts of the day
day_part = pd.read_sql("""
SELECT 
  CASE 
    WHEN CAST(Time/3600 AS INT) % 24 BETWEEN 0 AND 5 THEN 'Night'
    WHEN CAST(Time/3600 AS INT) % 24 BETWEEN 6 AND 11 THEN 'Morning'
    WHEN CAST(Time/3600 AS INT) % 24 BETWEEN 12 AND 17 THEN 'Afternoon'
    ELSE 'Evening'
  END AS time_of_day,
  COUNT(*) AS total_trans,
  SUM(CASE WHEN Class = 1 THEN 1 ELSE 0 END) AS fraud_trans
FROM creditcard
GROUP BY time_of_day""",conn)
day_part

,time_of_day,total_trans,fraud_trans
0,Afternoon,96435,134
1,Evening,93526,116
2,Morning,70912,118
3,Night,23934,124


In [9]:
# Comparing Amounts 
amount = pd.read_sql( """
SELECT Class,
MIN(Amount) AS min_amount,
MAX(Amount) AS max_amount,
ROUND(AVG(Amount), 2) AS average_amount
FROM creditcard
GROUP BY Class;""", conn)
amount

,Class,min_amount,max_amount,average_amount
0,0,0.0000,25691.1600,88.2900
1,1,0.0000,2125.8700,122.2100


In [14]:
# Risk Segmentation Analysis

risk_analysis = pd.read_sql("""
WITH risk_scoring AS (
    SELECT *,
        CASE WHEN Amount >= 1000 THEN 2 
             WHEN Amount >= 500 THEN 1 
             ELSE 0 END as amount_risk,
        CASE WHEN FLOOR(Time/3600) % 24 BETWEEN 0 AND 5 THEN 2
             WHEN FLOOR(Time/3600) % 24 BETWEEN 6 AND 8 THEN 1 
             ELSE 0 END as time_risk,
        (CASE WHEN Amount >= 1000 THEN 2 WHEN Amount >= 500 THEN 1 ELSE 0 END +
         CASE WHEN FLOOR(Time/3600) % 24 BETWEEN 0 AND 5 THEN 2 WHEN FLOOR(Time/3600) % 24 BETWEEN 6 AND 8 THEN 1 ELSE 0 END) as total_risk_score
    FROM creditcard
)
SELECT 
    CASE 
        WHEN total_risk_score >= 3 THEN 'High Risk'
        WHEN total_risk_score = 2 THEN 'Medium Risk'
        WHEN total_risk_score = 1 THEN 'Low Risk'
        ELSE 'Minimal Risk'
    END as risk_category,
    COUNT(*) as transaction_count,
    SUM(Class) as fraud_count,
    ROUND(AVG(Class) * 100, 2) as fraud_rate_percent,
    ROUND(AVG(Amount), 2) as avg_amount,
    ROUND(SUM(CASE WHEN Class = 1 THEN Amount ELSE 0 END), 2) as total_fraud_loss
FROM risk_scoring
GROUP BY 
    CASE 
        WHEN total_risk_score >= 3 THEN 'High Risk'
        WHEN total_risk_score = 2 THEN 'Medium Risk'
        WHEN total_risk_score = 1 THEN 'Low Risk'
        ELSE 'Minimal Risk'
    END
ORDER BY fraud_rate_percent DESC;
""", conn)
risk_analysis

,risk_category,transaction_count,fraud_count,fraud_rate_percent,avg_amount,total_fraud_loss
0,High Risk,707,7,0.9900,1181.0100,6325.5500
1,Medium Risk,26574,126,0.4700,229.8200,16065.3200
2,Low Risk,26649,58,0.2200,182.8200,16124.7600
3,Minimal Risk,230877,301,0.1300,57.8200,21612.3400


In [20]:
# Time Analysis
time = pd.read_sql("""
SELECT 
    CASE 
        WHEN FLOOR(Time/3600) % 24 BETWEEN 0 AND 3 THEN 'Late Night (0-3)'
        WHEN FLOOR(Time/3600) % 24 BETWEEN 4 AND 7 THEN 'Early Morning (4-7)'
        WHEN FLOOR(Time/3600) % 24 BETWEEN 8 AND 11 THEN 'Morning (8-11)'
        WHEN FLOOR(Time/3600) % 24 BETWEEN 12 AND 15 THEN 'Afternoon (12-15)'
        WHEN FLOOR(Time/3600) % 24 BETWEEN 16 AND 19 THEN 'Evening (16-19)'
        ELSE 'Night (20-23)'
    END as time_period,
    COUNT(*) as number_of_transactions,
    SUM(Class) as fraud_count,
    ROUND(AVG(Class) * 100, 2) as fraud_rate,
    ROUND(AVG(Amount), 2) as average_transaction_amount,
    ROUND(AVG(CASE WHEN Class = 1 THEN Amount END), 2) as average_fraud_amount,
    ROUND(SUM(Amount), 2) as total_amount,
    ROUND(SUM(CASE WHEN Class = 1 THEN Amount ELSE 0 END), 2) as fraud_amount,
    ROUND(SUM(CASE WHEN Class = 1 THEN Amount ELSE 0 END) / SUM(Amount) * 100, 2) as fraud_loss_percent
FROM creditcard
GROUP BY 
    CASE 
        WHEN FLOOR(Time/3600) % 24 BETWEEN 0 AND 3 THEN 'Late Night (0-3)'
        WHEN FLOOR(Time/3600) % 24 BETWEEN 4 AND 7 THEN 'Early Morning (4-7)'
        WHEN FLOOR(Time/3600) % 24 BETWEEN 8 AND 11 THEN 'Morning (8-11)'
        WHEN FLOOR(Time/3600) % 24 BETWEEN 12 AND 15 THEN 'Afternoon (12-15)'
        WHEN FLOOR(Time/3600) % 24 BETWEEN 16 AND 19 THEN 'Evening (16-19)'
        ELSE 'Night (20-23)'
    END
ORDER BY fraud_rate DESC;
""", conn)
time

,time_period,number_of_transactions,fraud_count,fraud_rate,average_transaction_amount,average_fraud_amount,total_amount,fraud_amount,fraud_loss_percent
0,Late Night (0-3),18735,90,0.4800,61.0700,87.2400,1144206.5500,7851.2800,0.6900
1,Early Morning (4-7),16543,66,0.4000,65.2700,101.9700,1079720.7100,6729.9700,0.6200
2,Evening (16-19),65307,103,0.1600,89.2500,158.6300,5828831.5500,16338.7200,0.2800
3,Morning (8-11),59568,86,0.1400,104.2300,123.3300,6209039.0600,10606.7400,0.1700
4,Afternoon (12-15),63816,83,0.1300,102.6400,138.2700,6550374.9100,11476.2300,0.1800
5,Night (20-23),60838,64,0.1100,71.5100,111.3300,4350417.2300,7125.0300,0.1600


In [21]:
# Statistical Distribution Analysis
statistical_analysis = pd.read_sql("""
WITH amount_ranges AS (
    SELECT 
        Class,
        CASE 
            WHEN Amount = 0 THEN 'Zero Amount'
            WHEN Amount <= 25 THEN '$0-25'
            WHEN Amount <= 50 THEN '$25-50'
            WHEN Amount <= 100 THEN '$50-100'
            WHEN Amount <= 200 THEN '$100-200'
            WHEN Amount <= 500 THEN '$200-500'
            WHEN Amount <= 1000 THEN '$500-1000'
            WHEN Amount <= 2000 THEN '$1000-2000'
            ELSE '$2000+'
        END as amount_range,
        Amount
    FROM creditcard
)
SELECT 
    amount_range,
    COUNT(*) as total_count,
    SUM(CASE WHEN Class = 0 THEN 1 ELSE 0 END) as legitimate_count,
    SUM(CASE WHEN Class = 1 THEN 1 ELSE 0 END) as fraud_count,
    ROUND(AVG(CASE WHEN Class = 1 THEN 1.0 ELSE 0.0 END) * 100, 2) as fraud_rate,
    ROUND(AVG(Amount), 2) as avg_amount_in_range,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM creditcard), 2) as percent_of_total_transactions
FROM amount_ranges
GROUP BY amount_range
ORDER BY MIN(CASE WHEN amount_range = 'Zero Amount' THEN 0 ELSE 1 END), MIN(Amount);
""", conn)
statistical_analysis

,amount_range,total_count,legitimate_count,fraud_count,fraud_rate,avg_amount_in_range,percent_of_total_transactions
0,Zero Amount,1825,1798,27,1.4800,0.0000,0.6400
1,$0-25,148529,148280,249,0.1700,8.2900,52.1500
2,$25-50,40691,40661,30,0.0700,37.2500,14.2900
3,$50-100,37254,37198,56,0.1500,73.3000,13.0800
4,$100-200,27671,27626,45,0.1600,143.1800,9.7200
5,$200-500,19695,19645,50,0.2500,310.1200,6.9200
6,$500-1000,6202,6176,26,0.4200,693.3800,2.1800
7,$1000-2000,2264,2256,8,0.3500,1362.7500,0.7900
8,$2000+,676,675,1,0.1500,3298.8700,0.2400


In [22]:
# Summary
summary = pd.read_sql("""
SELECT 
    'Overall Dataset' as metric_category,
    COUNT(*) as total_transactions,
    SUM(Class) as fraud_transactions,
    ROUND(AVG(Class) * 100, 3) as fraud_rate_percent,
    ROUND(SUM(Amount), 2) as total_transaction_value,
    ROUND(SUM(CASE WHEN Class = 1 THEN Amount ELSE 0 END), 2) as total_fraud_loss,
    ROUND(AVG(Amount), 2) as avg_transaction_amount,
    ROUND(AVG(CASE WHEN Class = 1 THEN Amount END), 2) as avg_fraud_amount
FROM creditcard

UNION ALL

SELECT 
    'High Value Transactions (>$1000)',
    COUNT(*),
    SUM(Class),
    ROUND(AVG(Class) * 100, 3),
    ROUND(SUM(Amount), 2),
    ROUND(SUM(CASE WHEN Class = 1 THEN Amount ELSE 0 END), 2),
    ROUND(AVG(Amount), 2),
    ROUND(AVG(CASE WHEN Class = 1 THEN Amount END), 2)
FROM creditcard
WHERE Amount >= 1000;
""", conn)
summary

,metric_category,total_transactions,fraud_transactions,fraud_rate_percent,total_transaction_value,total_fraud_loss,avg_transaction_amount,avg_fraud_amount
0,Overall Dataset,284807,492,0.1730,25162590.0100,60127.9700,88.3500,122.2100
1,High Value Transactions (>$1000),3069,9,0.2930,5444309.9800,13237.3300,1773.9700,1470.8100
